In [1]:
# Data processing
# ==============================================================================
import numpy as np
import pandas as pd

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
#from statsmodels.graphics.tsaplots import plot_acf
#from statsmodels.graphics.tsaplots import plot_pacf
#import plotly.graph_objects as go
#import plotly.io as pio
#import plotly.offline as poff
#pio.templates.default = "seaborn"t
#poff.init_notebook_mode(connected=True)
#plt.style.use('seaborn-v0_8-darkgrid')

# Modelling and Forecasting
# ==============================================================================
from xgboost import XGBRegressor
#import skforecast
from tqdm.notebook import tqdm

import utils
from feature_engineering import codify_date, codify_date_2, remove_outliers, get_X_y, covid_19, covid_19_2
from feature_engineering import add_weather, add_lag_and_rolling_features
from utils import handle_missing_values
from pathlib import Path
from lightgbm import LGBMRegressor


In [2]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection import TimeSeriesFold, OneStepAheadFold
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection import bayesian_search_forecaster_multiseries
from skforecast.feature_selection import select_features_multiseries
from skforecast.preprocessing import RollingFeatures
from skforecast.plot import set_dark_theme
from skforecast.exceptions import OneStepAheadValidationWarning
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_parquet(Path("data") / "train.parquet")
data = codify_date_2(data)
data = remove_outliers(data)
data = covid_19_2(data)
data = add_weather(data)
data = handle_missing_values(data, "linear")
data = data.drop(columns=["counter_id", "site_id", "site_name", 
                                "counter_installation_date", 
                              "coordinates", "counter_technical_id",
                              "latitude", "longitude", "date", "bike_count"])
data.head()

/Users/joaosilva/Current/École Polytechnique/Python for Data Science/Project/bike_counters/feature_engineering.py:41: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby(["counter_name", "date_truncated"])
/Users/joaosilva/Current/École Polytechnique/Python for Data Science/Project/bike_counters/feature_engineering.py:72: DtypeWarning: Columns (2,3,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  covid_19_index = pd.read_csv(Path("data") / "Covid_19_Index.csv")


Columns with missing values and their counts:
rr1         1326
ht_neige    7232
raf10       1650
etat_sol    8022
dtype: int64


,StringencyIndex_Average,counter_name,log_bike_count,datetime,year,month,day,day_of_week,hour,is_weekend,IsHoliday,t,rr1,u,ht_neige,raf10,ff,ww,etat_sol,tend
0,46.76,Face au 8 avenue de la porte de Charenton NO-SE,1.098612,2020-09-01 01:00:00,2020,9,1,1,1,False,False,285.75,0.0,81,0.0,2.4,1.6,1,0.0,-10
1,46.76,Voie Georges Pompidou NE-SO,0.000000,2020-09-01 01:00:00,2020,9,1,1,1,False,False,285.75,0.0,81,0.0,2.4,1.6,1,0.0,-10
2,46.76,20 Avenue de Clichy NO-SE,2.079442,2020-09-01 01:00:00,2020,9,1,1,1,False,False,285.75,0.0,81,0.0,2.4,1.6,1,0.0,-10
3,46.76,Pont des Invalides S-N,0.000000,2020-09-01 01:00:00,2020,9,1,1,1,False,False,285.75,0.0,81,0.0,2.4,1.6,1,0.0,-10
4,46.76,39 quai François Mauriac NO-SE,1.098612,2020-09-01 01:00:00,2020,9,1,1,1,False,False,285.75,0.0,81,0.0,2.4,1.6,1,0.0,-10


In [4]:
data.dtypes

StringencyIndex_Average           float64
counter_name                     category
log_bike_count                    float64
datetime                   datetime64[ns]
year                                int32
month                               int32
day                                 int32
day_of_week                         int32
hour                                int32
is_weekend                           bool
IsHoliday                            bool
t                                 float64
rr1                               float64
u                                   int64
ht_neige                          float64
raf10                             float64
ff                                float64
ww                                  int64
etat_sol                          float64
tend                                int64
dtype: object

In [5]:
series = data[["counter_name", "datetime", "log_bike_count"]]
exog = data[["counter_name", "datetime", 't', 'rr1', 'u', 'ht_neige', 'raf10', 'ff', 'ww', 'etat_sol', 'tend', "StringencyIndex_Average", "year", "month", "hour", "day_of_week", "IsHoliday"]]

In [6]:
categorical_columns = ["year", "month", "day", "hour", "day_of_week", "IsHoliday"]
def cyclic_transform(df, col, period):
    df[f"{col}_sin"] = np.sin(2 * np.pi * df[col] / period)
    df[f"{col}_cos"] = np.cos(2 * np.pi * df[col] / period)
    df = df.drop(columns=col)
    return df

def one_hot_encode(df, cols):
    encoder = OneHotEncoder(sparse_output=False, drop=None)
    encoded_array = encoder.fit_transform(df[cols])
    encoded_cols = encoder.get_feature_names_out(cols)
    df_encoded = pd.DataFrame(encoded_array, columns=encoded_cols, index=df.index)
    df_encoded = df_encoded.astype(float)  # Ensure encoded features are floats
    df = df.drop(columns=cols).join(df_encoded)
    return df

# Apply sine and cosine transformations
exog["tend"] = exog["tend"].astype(float)
exog["u"] = exog["u"].astype(float)
exog["ww"] = exog["ww"].astype(float)
exog["year"] = exog["year"].astype(float)
exog["IsHoliday"] = exog["IsHoliday"].astype(float)
#exog_train = cyclic_transform(exog_train, "month", 12)
exog = cyclic_transform(exog, "hour", 24)
#exog_train = cyclic_transform(exog_train, "day_of_week", 7)
exog = one_hot_encode(exog, ["month", "day_of_week"])
exog.head()

/var/folders/b3/l_t9dzy13rz2smp7vyx9352h0000gn/T/ipykernel_20056/3303039429.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exog["tend"] = exog["tend"].astype(float)
/var/folders/b3/l_t9dzy13rz2smp7vyx9352h0000gn/T/ipykernel_20056/3303039429.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exog["u"] = exog["u"].astype(float)
/var/folders/b3/l_t9dzy13rz2smp7vyx9352h0000gn/T/ipykernel_20056/3303039429.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

,counter_name,datetime,t,rr1,u,ht_neige,raf10,ff,ww,etat_sol,...,month_10,month_11,month_12,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
0,Face au 8 avenue de la porte de Charenton NO-SE,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Voie Georges Pompidou NE-SO,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,20 Avenue de Clichy NO-SE,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Pont des Invalides S-N,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,39 quai François Mauriac NO-SE,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [7]:
exog.head()

,counter_name,datetime,t,rr1,u,ht_neige,raf10,ff,ww,etat_sol,...,month_10,month_11,month_12,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
0,Face au 8 avenue de la porte de Charenton NO-SE,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Voie Georges Pompidou NE-SO,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,20 Avenue de Clichy NO-SE,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Pont des Invalides S-N,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,39 quai François Mauriac NO-SE,2020-09-01 01:00:00,285.75,0.0,81.0,0.0,2.4,1.6,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
series_dict = series_long_to_dict(
    data      = series,
    series_id = 'counter_name',
    index     = 'datetime',
    values    = 'log_bike_count',
    freq      = 'H'
)

exog_dict = exog_long_to_dict(
    data      = exog,
    series_id = 'counter_name',
    index     = 'datetime',
    freq      = 'H'
)

/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:418: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  original_sizes = data.groupby(series_id).size()
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:420: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for k, v in data.groupby(series_id):
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:421: FutureWarning: 'H' is deprecated and will be removed

In [9]:
# Partition data in train and test
# ==============================================================================
end_train = '2021-08-09 23:00:00'
series_dict_train = {k: v.loc[: end_train,] for k, v in series_dict.items()}
exog_dict_train   = {k: v.loc[: end_train,] for k, v in exog_dict.items()}
series_dict_test  = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test    = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

In [10]:
# Description of each series
# ==============================================================================
for k in series_dict.keys():
    print(f"{k}:")
    try:
        print(
            f"\tTrain: len={len(series_dict_train[k])}, {series_dict_train[k].index[0]}"
            f" --- {series_dict_train[k].index[-1]} "
            f" (missing={series_dict_train[k].isnull().sum()})"
        )
    except:
        print(f"\tTrain: len=0")
    try:
        print(
            f"\tTest : len={len(series_dict_test[k])}, {series_dict_test[k].index[0]}"
            f" --- {series_dict_test[k].index[-1]} "
            f" (missing={series_dict_test[k].isnull().sum()})"
        )
    except:
        print(f"\tTest : len=0")


152 boulevard du Montparnasse E-O:
	Train: len=8231, 2020-09-01 01:00:00 --- 2021-08-09 23:00:00  (missing=703)
	Test : len=745, 2021-08-09 23:00:00 --- 2021-09-09 23:00:00  (missing=0)
152 boulevard du Montparnasse O-E:
	Train: len=8231, 2020-09-01 01:00:00 --- 2021-08-09 23:00:00  (missing=703)
	Test : len=745, 2021-08-09 23:00:00 --- 2021-09-09 23:00:00  (missing=0)
18 quai de l'Hôtel de Ville NO-SE:
	Train: len=8231, 2020-09-01 01:00:00 --- 2021-08-09 23:00:00  (missing=1)
	Test : len=745, 2021-08-09 23:00:00 --- 2021-09-09 23:00:00  (missing=0)
18 quai de l'Hôtel de Ville SE-NO:
	Train: len=8231, 2020-09-01 01:00:00 --- 2021-08-09 23:00:00  (missing=1)
	Test : len=745, 2021-08-09 23:00:00 --- 2021-09-09 23:00:00  (missing=0)
20 Avenue de Clichy NO-SE:
	Train: len=8231, 2020-09-01 01:00:00 --- 2021-08-09 23:00:00  (missing=1921)
	Test : len=745, 2021-08-09 23:00:00 --- 2021-09-09 23:00:00  (missing=0)
20 Avenue de Clichy SE-NO:
	Train: len=8231, 2020-09-01 01:00:00 --- 2021-08-09 2

In [11]:
# Exogenous variables for each series
# ==============================================================================
for k in series_dict.keys():
    print(f"{k}:")
    try:
        print(f"\t{exog_dict[k].columns.to_list()}")
    except:
        print(f"\tNo exogenous variables")

152 boulevard du Montparnasse E-O:
	['t', 'rr1', 'u', 'ht_neige', 'raf10', 'ff', 'ww', 'etat_sol', 'tend', 'StringencyIndex_Average', 'year', 'IsHoliday', 'hour_sin', 'hour_cos', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6']
152 boulevard du Montparnasse O-E:
	['t', 'rr1', 'u', 'ht_neige', 'raf10', 'ff', 'ww', 'etat_sol', 'tend', 'StringencyIndex_Average', 'year', 'IsHoliday', 'hour_sin', 'hour_cos', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6']
18 quai de l'Hôtel de Ville NO-SE:
	['t', 'rr1', 'u', 'ht_neige', 'raf10', 'ff', 'ww', 'etat_sol', 'tend', 'StringencyIndex_Average', 

In [ ]:
# Fit forecaster
# ==============================================================================
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

window_features = RollingFeatures(stats=['mean', 'mean'], window_sizes=[24, 168])
forecaster = ForecasterRecursiveMultiSeries(
                regressor          = HistGradientBoostingRegressor(random_state=123),
                lags               = [1, 24, 168],
                window_features    = window_features,
                encoding           = "ordinal",
                dropna_from_series = False
            )

forecaster.fit(series=series_dict_train, exog=exog_dict_train, suppress_warnings=True)
forecaster

============================== 
ForecasterRecursiveMultiSeries 
============================== 
Regressor: LGBMRegressor 
Lags: [  1  24 168] 
Window features: ['roll_mean_24', 'roll_mean_168'] 
Window size: 168 
Series encoding: ordinal 
Series names (levels): 
    152 boulevard du Montparnasse E-O, 152 boulevard du Montparnasse O-E, 18 quai de
    l'Hôtel de Ville NO-SE, 18 quai de l'Hôtel de Ville SE-NO, 20 Avenue de
    Clichy NO-SE, 20 Avenue de Clichy SE-NO, 254 rue de Vaugirard NE-SO, 254 rue
    de Vaugirard SO-NE, 27 quai de la Tournelle NO-SE, 27 quai de la Tournelle
    SE-NO, 28 boulevard Diderot E-O, 28 boulevard Diderot O-E, 36 quai de
    Grenelle NE-SO, 36 quai de Grenelle SO-NE, 38 rue Turbigo NE-SO, 38 rue
    Turbigo SO-NE, 39 quai François Mauriac NO-SE, 39 quai François Mauriac SE-
    NO, 6 rue Julia Bartet NE-SO, 6 rue Julia Bartet SO-NE, 67 boulevard
    Voltaire SE-NO, 90 Rue De Sèvres NE-SO, 90 Rue De Sèvres SO-NE, Face 104 rue
    d'Aubervilliers N-S, Face 104 rue d'Aubervilliers S-N, ..., Face au 48 quai
    de la marne NE-SO, Face au 48 quai de la marne SO-NE, Face au 70 quai de
    Bercy N-S, Face au 70 quai de Bercy S-N, Face au 8 avenue de la porte de
    Charenton NO-SE, Face au 8 avenue de la porte de Charenton SE-NO, Pont
    Charles De Gaulle NE-SO, Pont Charles De Gaulle SO-NE, Pont de Bercy NE-SO,
    Pont de Bercy SO-NE, Pont de la Concorde S-N, Pont des Invalides N-S, Pont
    des Invalides S-N, Quai d'Orsay E-O, Quai d'Orsay O-E, Totem 64 Rue de
    Rivoli E-O, Totem 64 Rue de Rivoli O-E, Totem 73 boulevard de Sébastopol
    N-S, Totem 73 boulevard de Sébastopol S-N, Totem 85 quai d'Austerlitz NO-SE,
    Totem 85 quai d'Austerlitz SE-NO, Totem Cours la Reine E-O, Totem Cours la
    Reine O-E, Voie Georges Pompidou NE-SO, Voie Georges Pompidou SO-NE 
Exogenous included: True 
Exogenous names: 
    t, rr1, u, ht_neige, raf10, ff, ww, etat_sol, tend, StringencyIndex_Average,
    year, IsHoliday, hour_sin, hour_cos, month_1, month_2, month_3, month_4,
    month_5, month_6, month_7, month_8, month_9, month_10, month_11, month_12,
    day_of_week_0, day_of_week_1, day_of_week_2, day_of_week_3, day_of_week_4,
    day_of_week_5, day_of_week_6 
Transformer for series: None 
Transformer for exog: None 
Weight function included: False 
Series weights: None 
Differentiation order: None 
Training range: 
    '152 boulevard du Montparnasse E-O': ['2020-09-01 01:00:00', '2021-08-09
    23:00:00'], '152 boulevard du Montparnasse O-E': ['2020-09-01 01:00:00',
    '2021-08-09 23:00:00'], '18 quai de l'Hôtel de Ville NO-SE': ['2020-09-01
    01:00:00', '2021-08-09 23:00:00'], '18 quai de l'Hôtel de Ville SE-NO':
    ['2020-09-01 01:00:00', '2021-08-09 23:00:00'], '20 Avenue de Clichy NO-SE':
    ['2020-09-01 01:00:00', '2021-08-09 23:00:00'], ..., 'Totem 85 quai
    d'Austerlitz SE-NO': ['2020-09-01 01:00:00', '2021-08-09 23:00:00'], 'Totem
    Cours la Reine E-O': ['2020-09-01 01:00:00', '2021-08-09 23:00:00'], 'Totem
    Cours la Reine O-E': ['2020-09-01 01:00:00', '2021-08-09 23:00:00'], 'Voie
    Georges Pompidou NE-SO': ['2020-09-01 01:00:00', '2021-08-09 23:00:00'],
    'Voie Georges Pompidou SO-NE': ['2020-09-01 01:00:00', '2021-08-09
    23:00:00'] 
Training index type: DatetimeIndex 
Training index frequency: h 
Regressor parameters: 
    {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0,
    'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1,
    'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0,
    'n_estimators': 100, 'n_jobs': None, 'num_leaves': 31, 'objective': None,
    'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0,
    'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbosity': -1} 
fit_kwargs: {} 
Creation date: 2024-12-11 15:56:56 
Last fit date: 2024-12-11 15:56:58 
Skforecast version: 0.14.0 
Python version: 3.12.4 
Forecaster id: None

In [13]:
predictions = forecaster.predict(steps=1020, exog=exog_dict_test)

/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/utils/utils.py:966: MissingValuesWarning: `exog` for series '152 boulevard du Montparnasse E-O' doesn't have as many values as steps predicted, 1020. Missing values are filled with NaN. Most of machine learning models do not allow missing values. Prediction method may fail. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/utils/utils.py:1036: MissingValuesWarning: To make predictions `exog` for series '152 boulevard du Montparnasse E-O' must start one step ahead of `last_window`. Missing values are filled with NaN.
    `last_window` ends at : 2021-08-09 23:00:00.
    `exog` for series '152 boulevard du Montparnasse E-O' starts at : 2021-08-09 23:00:00.
     Expected index       : 2021-08-10 00:00:00. 
 You can suppress this warning us

In [14]:
predictions = predictions.reset_index(names='datetime').melt(id_vars=['datetime'], var_name='counter_name', value_name='log_bike_count')

series_test = series.query(f"datetime > '{end_train}'")

merged_df = series_test.merge(predictions, on=['counter_name', 'datetime'], how='left')

In [15]:
from sklearn.metrics import root_mean_squared_error
root_mean_squared_error(merged_df["log_bike_count_x"], merged_df["log_bike_count_y"])

np.float64(0.571707351202753)

In [16]:
merged_df[["datetime", "log_bike_count_y"]]

,datetime,log_bike_count_y
0,2021-08-10 00:00:00,0.545923
1,2021-08-10 00:00:00,1.245902
2,2021-08-10 00:00:00,1.158357
3,2021-08-10 00:00:00,0.662539
4,2021-08-10 00:00:00,0.551348
...,...,...
41659,2021-09-09 23:00:00,1.270191
41660,2021-09-09 23:00:00,1.264279
41661,2021-09-09 23:00:00,1.899439
41662,2021-09-09 23:00:00,2.496556
